In [1]:
import pandas as pd

In [4]:
data_petshop = pd.read_csv('1655023Pet shop.csv')
data_petshop.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size
0,1,67648,health,off_brand,3.5,12,33.0,42.3,852
1,2,120421,other,off_brand,4.5,10,16.0,54.3,1045
2,3,95242,toys,off_brand,2.9,6,25.0,88.2,1461
3,4,105632,health,off_brand,2.6,8,NaN,5.0,1579
4,5,221627,toys,off_brand,4.9,11,25.0,57.0,2643


In [ ]:
#assignment 1 (15%)
#

In [8]:
pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 46.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 6.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import statsmodels.api as sm

In [12]:
# Turn categorial into dummies
data_petshop_with_dummies = pd.get_dummies(data_petshop, drop_first=True)
# remove missing values
data_petshop_clean = data_petshop_with_dummies.dropna()
